# Book Recommendations.

In [2]:
import yaml
import pandas as pd
from BookRecommender import BookRecommender

In [ ]:
%%time

with open("books.yaml", "r") as f:
    books = yaml.safe_load(f)
    

rec = BookRecommender(books)

In [6]:
%%time
rec("a story in space", top_n=4)

CPU times: user 96.7 ms, sys: 6.44 ms, total: 103 ms
Wall time: 109 ms


[['The Space Between Worlds',
  'An outsider who can travel between worlds discovers a secret that threatens her new home and her fragile place in it, in a stunning sci-fi debut that’s both a cross-dimensional adventure and a powerful examination of identity, privilege, and belonging. Multiverse travel is finally possible, but there’s just one catch No one can visit a world where their counterpart is still alive. Enter Cara, whose parallel selves happen to be exceptionally good at dying—from disease, turf wars, or vendettas they couldn’t outrun. Cara’s life has been cut short on 372 worlds in total. On this Earth, however, Cara has survived. Identified as an outlier and therefore a perfect candidate for multiverse travel, Cara is plucked from the dirt of the wastelands. Now she has a nice apartment on the lower levels of the wealthy and walled-off Wiley City. She works—and shamelessly flirts—with her enticing yet aloof handler, Dell, as the two women collect off-world data for the Eldr